In [2]:
from cgeniepy.ecology import EcoModel
import pandas as pd
from cgeniepy.grid import GENIE_lat
import numpy as np
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
from cgeniepy.array import GenieArray

foram = ["bn", "bs", "sn", "ss"]

lat = GENIE_lat() * -1
lat_array = np.broadcast_to(lat, (36, 36)).T

def relative_abundance(model, foram):
    foram_biomass = model.get_foram(foram).isel(time=-1).array
    foram_biomass_total = model.get_foram(['bn','bs','sn','ss']).array.sum(dim=['variable']).isel(time=-1)
    ra = foram_biomass/foram_biomass_total
    ## if foram_biomass is very small, make ra to 0
    ## 0.845 μg per foram cell
    ## 1 foram cell / m3 => 0.845 μg / m3 => 0.845e-6 g / m3 =>  0.845e-6/12 mol/ m3
    ## => 0.845e-3/12 mmol/ m3
    threshold = 0.845e-3/12
    ra = ra.where(foram_biomass_total > threshold, ra, 0)

    ## return GenieArray
    genie_array = GenieArray()
    genie_array.array = ra
    return genie_array

mod_paths = [
              "../model/muffin.CBE.worlg4.BASESFeTDTL.SPIN",
              "../model/muffin.CBE.GIteiiva.BASESFeTDTL_rb.SPIN",
            "../model/muffin.CBE.worlg4.BASESFeTDTL.historical",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.1p5deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.2deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.3deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.4deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2.5xCO2"
             ]

export_files= ["../model/model_drived/pi_foramecogenie.csv",
               "../model/model_drived/lgm_foramecogenie.csv",
               "../model/model_drived/historical_foramecogenie.csv",
               "../model/model_drived/future1p5_foramecogenie.csv",
                "../model/model_drived/future2_foramecogenie.csv",
                "../model/model_drived/future3_foramecogenie.csv",
                "../model/model_drived/future4_foramecogenie.csv",
               "../model/model_drived/2.5xCO2_foramecogenie.csv"
               ]


for path, csv_name in zip(mod_paths, export_files):
    mod = EcoModel(path)
    bn_c = relative_abundance(mod, "bn").array.values.ravel()
    bs_c = relative_abundance(mod, "bs").array.values.ravel()
    sn_c = relative_abundance(mod, "sn").array.values.ravel()
    ss_c = relative_abundance(mod, "ss").array.values.ravel()    

    sst = mod.get_var("ocn_sur_temp").isel(time=-1).array.values.ravel()
    sal = mod.get_var("ocn_sur_sal").isel(time=-1).array.values.ravel()
    po4 = mod.get_var("ocn_sur_PO4").isel(time=-1).array.values.ravel()
    light = mod.get_var("phys_fxsw").isel(time=-1).array.values.ravel()
    chl_total = mod.get_var("eco2D_Plankton_C_Total").isel(time=-1).array.values.ravel()
    
    fe = mod.get_var("ocn_sur_TDFe").isel(time=-1).array.values.ravel()
    mld = mod.get_var("phys_MLD").isel(time=-1).array.values.ravel()

    # masked!
    #random_var = mod.select_var("ocn_sur_temp").isel(time=-1).array.values
    #mask = np.where(np.isnan(random_var), np.nan, 1)
    ## masked lat
    #masked_lat = lat_array * mask

    #phys_w = mod.select_var("phys_w").isel(time=-1, zt=1).array
    #upwelling_discrete = np.where(phys_w >0, 1, 0)

    # temperature gradient between 283 m and 80 m
    #therm_gradient = mod.get_var("phys_w").isel(time=-1, zt=3).array - mod.get_var("phys_w").isel(time=-1, zt=1).array

    ## create data frame
    df = pd.DataFrame({"bn": bn_c, "bs": bs_c, "sn": sn_c, "ss": ss_c,                    
                       "sst": sst, "sal": sal, "po4": po4, "light": light,
                       "chl_total": chl_total,"fe": fe, "mld": mld,
                       #"therm_gradient": therm_gradient,
                       #"upwelling": upwelling_discrete.ravel(),
                       #"masked_lat": masked_lat.ravel()
                       })    

    # drop na
    df = df.dropna()
    #df = df.query('bn > 0 & bs > 0 & sn > 0 & ss > 0')
    #df = df.query('bn < 1 & bs < 1 & sn < 1 & ss < 1')
    df.to_csv(csv_name)

No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
No gemflag is provided, assuming the model includes biogem and ecogem
